In [ ]:
# Install dependencies
!pip install transformers datasets torch pdfplumber

import torch
import pdfplumber
from transformers import T5Tokenizer, T5ForConditionalGeneration, EncoderDecoderCache
from datasets import load_dataset
from google.colab import files
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

# Load the dataset
dataset = load_dataset("SHASWATSINGH3101/THE_BHARATIYA_NYAYA_SANHITA2023_summarize", split="train")

# Load the tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Convert dataset to PyTorch format
class LegalDataset(Dataset):
    def __init__(self, dataset):
        self.texts = dataset["text"]
        self.summaries = dataset["summary"]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = tokenizer("summarize: " + self.texts[idx], max_length=512, truncation=True, padding="max_length", return_tensors="pt")
        labels = tokenizer(self.summaries[idx], max_length=150, truncation=True, padding="max_length", return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": labels["input_ids"].squeeze(0)
        }

# Convert dataset
tokenized_dataset = LegalDataset(dataset)

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_dataloader = DataLoader(tokenized_dataset, shuffle=True, batch_size=4)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Fine-tuning loop (optional)
epochs = 2
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        batch = {key: val.to(device) for key, val in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} completed with loss: {total_loss / len(train_dataloader)}")

# Save trained model
model.save_pretrained("legal_summarizer")
tokenizer.save_pretrained("legal_summarizer")

# Function to summarize input text with updated past_key_values handling
def summarize_text(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Updated past_key_values handling
    past_key_values = None
    if hasattr(model, "past_key_values"):
        past_key_values = EncoderDecoderCache.from_legacy_cache(model.past_key_values)

    summary_ids = model.generate(
        **inputs,
        max_length=150,
        num_beams=4,
        early_stopping=True,
        past_key_values=past_key_values  # Corrected past key values usage
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Function to read text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    return text

# Upload file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Process the document
if file_name.endswith(".pdf"):
    document_text = extract_text_from_pdf(file_name)
else:
    with open(file_name, "r", encoding="utf-8") as f:
        document_text = f.read()

# Generate summary
summary = summarize_text(document_text)
print("\n=== Legal Document Summary ===\n")
print(summary)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

text_and_summary.json:   0%|          | 0.00/916k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1432 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1 completed with loss: 2.875180440242064
Epoch 2 completed with loss: 2.258356633799036


Saving vidit-mehra-updated.pdf to vidit-mehra-updated.pdf

=== Legal Document Summary ===

jlsr.thelawbrigade.com 114 CASE COMMENTARY: ADM JABALPUR v. SHIVKANT SHUKLA Written by Vidit Mehra 2nd Year of 3 Year LL.B. Student, Symbiosis Law School, Pune Citation: (1976) 2 SCC 521; AIR 1976 SC 1207 Bench: Ray, A.N. (Cj), Khanna, Hans Raj, Beg, M. Hameedullah


In [ ]:
# Install Gradio if not already installed
!pip install gradio

import gradio as gr
import nbformat
from nbconvert import PythonExporter
import os

# Function to extract the summary from the existing notebook
def extract_summary_code(notebook_path):
    with open(notebook_path, "r", encoding="utf-8") as f:
        notebook_data = nbformat.read(f, as_version=4)

    code_cells = [cell["source"] for cell in notebook_data["cells"] if cell["cell_type"] == "code"]
    full_code = "\n".join(code_cells)
    
    return full_code

# Run the existing code and return the summary
def process_document(file):
    if file is None:
        return "No file uploaded!"
    
    # Save uploaded file temporarily
    temp_path = "/content/temp_uploaded_file.txt"
    with open(temp_path, "wb") as f:
        f.write(file.read())

    # Extract the summary logic from the notebook
    notebook_code = extract_summary_code("project.ipynb")

    # Execute the extracted code
    exec_globals = {"input_file": temp_path}
    exec(notebook_code, exec_globals)

    # Retrieve the summary (assuming your notebook saves it in `summary` variable)
    summary = exec_globals.get("summary", "Summary not found in code execution.")
    
    return summary

# Create Gradio Interface
iface = gr.Interface(
    fn=process_document,
    inputs=gr.File(label="Upload a Document"),
    outputs=gr.Textbox(label="Generated Summary"),
    title="Document Summarizer",
    description="Upload a document, and it will be processed to generate a summary."
)

# Launch the interface
iface.launch(share=True)
